In [20]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [21]:
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
if not openai_key:
    raise ValueError("OPENAI_API_KEY not found in .env")

# 🚀 Initialize FastAPI
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["https://www.basemshaker.com", "http://127.0.0.1:5500"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# 📬 FastAPI schema
class Message(BaseModel):
    text: str


# 🔍 Load and split website content
loader = WebBaseLoader("https://www.basemshaker.com")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

# 🔎 Embed documents for retrieval
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 🔁 Create a retriever + QA chain
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=False)

In [27]:
qa_chain.invoke("What is the purpose of this website?")

{'query': 'What is the purpose of this website?',
 'result': "This website appears to be a personal portfolio showcasing the skills, experiences, and expertise of Basem Shaker, a Data Scientist and Machine Learning Engineer. The purpose seems to be to provide information about Basem's technical skills, work experience, education, and projects in order to attract potential opportunities, collaborations, or discussions with visitors who are interested in his background and work. It also serves as a platform for Basem to showcase his work and accomplishments in the fields of machine learning, robotics, automation, simulations, and more."}

In [ ]:
# 📡 FastAPI endpoint
@app.post("/chat")
def chat_endpoint(msg: Message):
    response = qa_chain.invoke(msg.text)
    return response["result"]